# Recurrent Neural Networks

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| Dennis G. Wilson | <a href="https://supaerodatascience.github.io/deep-learning/">https://supaerodatascience.github.io/deep-learning/</a>

In this class, we'll cover a new type of deep neural network layer: recurrent layers. These layer types share the common feature of having internal or **hidden state**. This makes them well-suited to sequential data, such as time series or text data. They are heavily used in Natural Language Processing, the subject of [the next class](https://supaerodatascience.github.io/deep-learning/NLP.html).

Today, we'll look at a simple example of time series forecasting. This family of algorithms can be used for a variety of prediction tasks, such as [predicting stock prices](https://arxiv.org/pdf/1911.13288.pdf) and [seizure prediction](https://www.sciencedirect.com/science/article/pii/S001048251830132X). We'll focus on the use of Recurrent Neural Networks in this context.

The data we'll use today is a set of EEG readings used to study seizures. You can find the data [here](https://physionet.org/content/chbmit/1.0.0/), and in the `data` directory there is a single patient recording `chb01_chb01_03.edf` which includes a seizure episode. We won't focus on seizure detection today but will instead try to predict neural activity.

These files are stored in the European Data Format, https://www.edfplus.info/. We'll use the python library `pyedflib` to read them.

In [ ]:
# pip install pyedflib

In [ ]:
import os
import pyedflib
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
file_name = os.path.join('data','chb01_chb01_03.edf')
f = pyedflib.EdfReader(file_name)

In [ ]:
n = f.signals_in_file
labels = f.getSignalLabels()
print('%d different signals: %s' % (n, str(labels)))

In [ ]:
fig, axs = plt.subplots(5, sharex=True, figsize=(18,5))
for i in range(5):
    signal = f.readSignal(i)
    axs[i].plot(signal[::256])

As we can see, this single recording contains 23 different signals from different sensors placed. We'll focus at predicting a single signal. Our first data processing step is normalizing the data. This is beneficial for recurrent neural network training, but does require domain knowledge. In this case, we know the physical limits of the sensors, so we can use that to normalize the data between $[-1, 1]$.

In [ ]:
signal = f.readSignal(0)
signal = 2 * (signal - f.physical_min(0)) / (f.physical_max(0) - f.physical_min(0)) - 1.0

Another preparation often done with timeseries data is to check if it is **stationary**, ie if the mean and variance change over time. To do this, we'll use the [Dickey-Fuller test](https://en.wikipedia.org/wiki/Dickey%E2%80%93Fuller_test). This test can be very costly to compute, so we'll downsample the data. There are 256 samples per second in this data, so we'll take one sample each second.

In [ ]:
import statsmodels.api as sm
p_value = sm.tsa.stattools.adfuller(signal[::256])[1]
p_value

The small value means that the timeseries **is stationary**. This isn't a necessary condition for LSTMs, but it will help training. When the timeseries is not stationary (`p_value > 0.05`), it's normal to instead predict the **difference** between timesteps, ie $$y_t - y_{t-1}$$ which can be calculated in numpy using `diff` and in pandas with `shift`.

Finally, we split our data into training and test sets, perserving order:

In [ ]:
s_max = 1000
data = np.zeros((5, s_max))
for i in range(5):
    signal = f.readSignal(i)
    signal = 2 * (signal - f.physical_min(i)) / (f.physical_max(i) - f.physical_min(i)) - 1.0
    data[i, :] = signal[:s_max]
input = torch.from_numpy(data[:3, :-1])
target = torch.from_numpy(data[:3, 1:])
test_input = torch.from_numpy(data[3:, :-1])
test_target = torch.from_numpy(data[3:, 1:])

In [ ]:
plt.figure(figsize=(30,10))
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
for i in range(3):
    plt.plot(input[i, :500], label='train')
for i in range(2):
    plt.plot(test_input[i, :500], label='test')
plt.legend();

We will start with a prediction which is
+ Univariate: we're using one signal to predict the future of that signal without considering other features. The opposite of this is multivariate, where multiple features are used in prediction (and multiple features can be simultaneously predicted)
+ 1-Step: we're predicting one step into the future, which can also be said as having a horizon of one sample. N-step or multi-step timeseries prediction is just an extension to this.

# Baselines

First we'll analyse the data and propose some simple baselines. We'll use **iterative prediction**, also known as walk-forward, to predict the next step at each timestep. The first baseline we'll use is called the naive baseline, which simply uses the previous value.

In [ ]:
train_signal = input[0]
test_signal = target[0]

In [ ]:
# walk-forward validation
history = np.append(train_signal, np.zeros(len(test_signal)))
predictions = np.zeros(len(test_signal))
for i in range(len(test_signal)):
    t = len(train_signal) + i
    # make prediction
    predictions[i] = history[t-1]
    # observation
    history[t] = test_signal[i]

In [ ]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(predictions, test_signal))

In [ ]:
plt.figure(figsize=(18,3))
plt.plot(test_signal[:512], label='test')
plt.plot(predictions[:512], label='prediction')
plt.legend();

For a horizon of 1, the naive baseline is pretty good. Let's see if the historical data beyond one timestep can be useful. We'll start with a windowed approach, using the average of the past `W=5` timesteps as our prediction.

In [ ]:
history = np.append(train_signal, np.zeros(len(test_signal)))
predictions = np.zeros(len(test_signal))
for i in range(len(test_signal)):
    t = len(train_signal) + i
    # make prediction
    predictions[i] = np.mean(history[t-5:t])
    # observation
    history[t] = test_signal[i]

In [ ]:
print(np.sqrt(mean_squared_error(predictions, test_signal)))
plt.figure(figsize=(18,3))
plt.plot(test_signal[:512], label='test')
plt.plot(predictions[:512], label='prediction')
plt.legend();

While this captures the general trend of some parts of the data, the naive baseline is still superior in terms of root mean squared error. Another approach is to exponentially decrease the dependency on past predictions, known as exponential smoothing. The rate by which we decrease past predictions is the parameter $\alpha$.

In [ ]:
def exponential_smoothing(alpha):
    history = np.append(train_signal, np.zeros(len(test_signal)))
    predictions = np.zeros(len(test_signal))
    for i in range(len(test_signal)):
        t = len(train_signal) + i
        # make prediction
        if i == 0:
            predictions[i] = history[t-1]
        else:    
            predictions[i] = alpha * history[t-1] + (1 - alpha) * predictions[i-1]
        # observation
        history[t] = test_signal[i]
    return predictions

In [ ]:
predictions_01 = exponential_smoothing(0.1)
print(np.sqrt(mean_squared_error(predictions_01, test_signal)))

In [ ]:
predictions_05 = exponential_smoothing(0.5)
np.sqrt(mean_squared_error(predictions_05, test_signal))

In [ ]:
plt.figure(figsize=(18,3))
plt.plot(test_signal[:512], label='test')
plt.plot(predictions_01[:512], label='prediction, alpha=0.1')
plt.plot(predictions_05[:512], label='prediction, alpha=0.5')
plt.legend();

While lower alpha values help predict the general trend of our data, their RMSE is worse than using very short history. So for this EEG data, we're still struggling to make good use of the historical data to predict future data. Instead of a single parameter for history decay, we'll instead use a recurrent neural network to inform our reliance on memory for prediction, and optimize the network parameters using Stochastic Gradient Descent.

# Recurrent Neural Networks

A simple recurrent neural network layer is very similar to a fully-connected feed-forward neural network layer; it has a set of weights $W_x$ mapping the previous layer $x$ to each neuron of the recurrent layer, a bias term for each neuron, and an activation function. However, a recurrent neural network also has state; specifically, each neuron connects to every other neuron in the same layer with a time delay of 1. This means that a recurrent neural network layer has a second weight matrix $W_s$ of size $n$x$n$, where $n$ is the number of neurons in the recurrent layer.

$s_t = \tanh(W_{x} x + b_{x}  +  W_{s} s_{t-1} + b_{s})$

One way to consider these recurrent connections is by representing the previous activation functions of the recurrent layer as a hidden state, and using that hidden state as input to the network:

<img src="img/rnn.png">

Let's first observe the behavior of a single RNN layer using the `RNNCell` class from PyTorch. The documentation is [here](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html).

In [ ]:
rnn = nn.RNNCell(1, 10)

In [ ]:
ins = torch.randn(1, 1)
hx = torch.zeros(1, 10)
ins, hx

In [ ]:
hx = rnn(ins, hx)
hx

As we can see, the output of the `RNNCell` is the new hidden state. In order to predict the next value, we'll use a `Linear` model to map from this hidden state to a single output value.

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn1 = nn.RNNCell(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        hidden = torch.zeros(input.size(0), self.hidden_size, dtype=torch.double)
        # predict over the different signals
        outputs = []
        for input_t in input.split(1, dim=1):
            hidden = self.rnn1(input_t, hidden)
            output = self.linear(hidden)
            outputs += [output]
        outputs = torch.cat(outputs, dim=1)
        return outputs, hidden

In [ ]:
n_hidden = 16
rnn = RNN(1, n_hidden, 1).double()

In [ ]:
with torch.no_grad():
    output, _ = rnn(input)
    y = output.detach().numpy()

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(input[0, 1:], label='real')
plt.plot(y[0, :-1], label='prediction');

This isn't very convincing, but remember that it is random weights. We'll soon get to training, but first let's increase the lookahead of our recurrent neural network. We'll try to predict 5 timesteps ahead.

The main difference with recurrent neural networks is that they depend on the previous state for the current state's computation. Instead of simply prediction $Y = f(x)$ as in feed-forward neural networks, recurrent networks do $Y_1 = f(x_1, f(x_0))$. Here's what an example "unrolled" RNN looks like, where the hidden state is carried over to next timestep.

<img src="img/unrolled.png">

<div class="alert alert-success">
Exercise 1
    
Include a look-ahead of `future` timesteps in your RNN model by completing the class below. Make sure to continuously update the `hidden` state.
</div>

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn1 = nn.RNNCell(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, input, future=0):
        hidden = torch.zeros(input.size(0), self.hidden_size, dtype=torch.double)
        outputs = []
        for input_t in input.split(1, dim=1):
            hidden = self.rnn1(input_t, hidden)
            output = self.linear(hidden)
            outputs += [output]
        outputs = torch.cat(outputs, dim=1)
        return outputs, hidden

In [ ]:
# %load solutions/1_future.py

In [ ]:
rnn = RNN(1, n_hidden, 1).double()
f = 5
with torch.no_grad():
    output, _ = rnn(input, future=f)
    y = output.detach().numpy()
plt.figure(figsize=(30,10))
plt.plot(input[0, f:], label='real')
plt.plot(y[0, :-f], label='prediction');

## Backpropagation through time

Our network doesn't do very well, but it's using random weights. In order to train it, we'll need to calcuate the gradient throughout the iterative process. This is known as **backpropagation through time**, and it relies on the computation of not just the current timestep, but all previous timesteps as well.

<img src="img/bptt.png">

In [ ]:
rnn = RNN(1, n_hidden, 1).double()
criterion = nn.MSELoss()
optimizer = optim.LBFGS(rnn.parameters(), lr=0.8)

In [ ]:
# training step through all data
def closure():
    optimizer.zero_grad()
    out, hidden = rnn(input, future=5)
    loss = criterion(out[:, :-10], target[:, 5:])
    print('loss:', loss.item())
    loss.backward()
    return loss

In [ ]:
optimizer.step(closure)

In [ ]:
with torch.no_grad():
    output, _ = rnn(input)
    y = output.detach().numpy()

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(input[1, 1:], label='real')
plt.plot(y[1, :-1], label='prediction');

<div class="alert alert-success">
Exercise 2:

Increase the look-ahead of your model during training. Make sure to align the `target` data with the output of your model (ie, advanced by `N` timesteps). How far ahead can your model predict with good accuracy?
</div>

In [ ]:
%load solutions/2_train.py

# Long Short-Term Memory

A more complex recurrent neural network, which actually predates much of the current era of deep learning, is called the Long Short-Term Memory unit. This unit has multiple different internal states which are transformed to retain only the pertinent information.

Gers, Felix A., Jürgen Schmidhuber, and Fred Cummins. "Learning to forget: Continual prediction with LSTM." (1999): 850-855.

<img src='img/lstm.png'>

$$
i_t = \sigma(W_{xi}x_t + W_{hi}h_{t-1} + W_{ci}c_{t-1}+b_i)\\
f_t = \sigma(W_{xf}x_t + W_{hf}h_{t-1} + W_{cf}c_{t-1}+b_f)\\
c_t = f_tc_{t-1}+i_t\tanh(W_{xc}x_t + W_{hc}h_{t-1} + b_c)\\
o_t = \sigma(W_{xo}x_t + W_{ho}h_{t-1} + W_{co}c_{t-1}+b_o)\\
h_t = o_t\tanh(c_t)
$$

<div class="alert alert-success">
Exercise 3

Modify your model from part exercise 2 to use the PyTorch `LSTMCell` classes. Be aware of the multiple return values from an LSTM - the hidden state and cell state. The documentation is [here](https://pytorch.org/docs/stable/generated/torch.nn.LSTMCell.html). Compare your test prediction error against the RNN model and against the naive baseline. 

</div>

In [ ]:
# %load solutions/3_lstm.py

In [ ]:
lstm = LSTM(1, n_hidden, 1).double()
criterion = nn.MSELoss()
optimizer = optim.LBFGS(lstm.parameters(), lr=0.8)

In [ ]:
# training step through all data
def closure():
    optimizer.zero_grad()
    out = lstm(input)
    loss = criterion(out, target)
    print('loss:', loss.item())
    loss.backward()
    return loss

In [ ]:
optimizer.step(closure)

In [ ]:
with torch.no_grad():
    outputs = lstm(input)
    y = outputs.detach().numpy()

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(input[1, 1:], label='real')
plt.plot(y[1, :-1], label='prediction');

<div class="alert alert-info">
Truncated BPTT

So far, we've been iterating through the entire dataset each epoch. In order to split the data into batches, what would need to be done? How could the hidden state of your recurrent networks be preserved between batches?
</div>

<div class="alert alert-success">
Bonus Exercise

Another recurrent layer type is the Gated Recurrent Unit. It was designed to solve the issue of vanishing gradients in LSTMs. [Here](https://arxiv.org/pdf/1412.3555.pdf) is an empirical study of the layer types. Try replacing your LSTM with a GRU to see if it changes your results.
</div>

If you'd like to know more about time-series prediction, [this notebook](https://github.com/marcopeix/stock-prediction/blob/master/Stock%20Prediction.ipynb) gives an example of more complex classical prediction models, notably Seasonal ARIMA.

In the next class, we'll see one of the most popular uses of LSTMs currently, in the context of [Natural Language Processing](https://arxiv.org/pdf/1810.04805.pdf). Before next class, please make sure you've installed `rasa`, `gensim`, and `nest-asyncio`.